In [1]:
from kan import *
import pytz
import time
import datetime
import torch.optim as optim
from data_process import data_process_without_norm
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import copy
import pandas as pd

df = data_process_without_norm()
df = df.drop("entsoe",axis=1)
loc_tz = pytz.timezone('Europe/Zurich')
split_date_train_ = loc_tz.localize(datetime.datetime(2016,1,1,0,0,0,0))
split_date_train = loc_tz.localize(datetime.datetime(2016,3,1,0,0,0,0))
split_date_test = loc_tz.localize(datetime.datetime(2016,3,15,0,0,0,0))

df_train_ = df.loc[(split_date_train_ < df.index)]
df_train = df_train_.loc[df_train_.index <= split_date_train].copy()
_temp_df = df.loc[split_date_test > df.index]
df_test = _temp_df.loc[_temp_df.index > split_date_train].copy()

# Input standardization
scaler_input = StandardScaler()
# scaler_input = MinMaxScalerr()
_temp_scaled_input_data = scaler_input.fit_transform(df_train.iloc[:,1:])
x_train_input = _temp_scaled_input_data

#Output standardization
scaler_output = StandardScaler()
# scaler_input = MinMaxScaler()
_temp_scaled_output_data = scaler_output.fit_transform(np.array(df_train.iloc[:,0]).reshape(-1,1))
x_train_label = _temp_scaled_output_data

#Test set standardization
x_test_input = scaler_input.transform(df_test.iloc[:,1:])
x_test_label = scaler_output.transform(np.array(df_test.iloc[:,0]).reshape(-1,1))

_temp_test_input = np.hstack((x_test_input[:,:8],x_test_input[:,-3:-1]))
_temp_test_label = x_test_label
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

_temp_train_input = np.hstack((x_train_input[:, :8], x_train_input[:, -3:-1]))
_temp_train_label = x_train_label

# 将数据转换为 PyTorch 张量
train_input_tensor = torch.tensor(_temp_train_input.astype(np.float32))
train_label_tensor = torch.tensor(_temp_train_label.astype(np.float32))

test_input_tensor = torch.tensor(_temp_test_input.astype(np.float32))
test_label_tensor = torch.tensor(_temp_test_label.astype(np.float32))
# 定义 k 折交叉验证
k = 5  # 例如，5折交叉验证
kf = KFold(n_splits=k, shuffle=True, random_state=42)


In [2]:
# CNN 模型
class CNN(nn.Module):
    def __init__(self, num_inputs, num_outputs, conv_out_channels=16, time_step=24):
        super(CNN, self).__init__()
        self.conv = nn.Conv1d(in_channels=num_inputs, out_channels=conv_out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(conv_out_channels * time_step, num_outputs)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # 调整输入形状为 (batch_size, num_inputs, time_step)
        x = self.conv(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)  # 展平
        x = self.fc(x)
        return x

# LSTM 模型
class LSTM(nn.Module):
    def __init__(self, num_inputs, num_outputs, lstm_hidden_units=16):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=num_inputs, hidden_size=lstm_hidden_units, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_units, num_outputs)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # 取最后一个时间步的输出
        x = self.fc(x)
        return x

# CNNLSTM 模型
class CNNLSTM(nn.Module):
    def __init__(self, num_inputs, num_outputs, conv_out_channels=16, lstm_hidden_units=16, time_step=24):
        super(CNNLSTM, self).__init__()
        self.conv = nn.Conv1d(in_channels=num_inputs, out_channels=conv_out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.lstm = nn.LSTM(input_size=conv_out_channels, hidden_size=lstm_hidden_units, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_units, num_outputs)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # 调整输入形状为 (batch_size, num_inputs, time_step)
        x = self.conv(x)
        x = self.relu(x)
        x = x.permute(0, 2, 1)  # 调整回 LSTM 所需的输入形状 (batch_size, time_step, conv_out_channels)
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # 取最后一个时间步的输出
        x = self.fc(x)
        return x

# Transformer 模型
class Transformer(nn.Module):
    def __init__(self, num_inputs, num_outputs, d_model=16, num_heads=2, num_layers=2):
        super(Transformer, self).__init__()
        self.embedding = nn.Linear(num_inputs, d_model)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        self.fc = nn.Linear(d_model, num_outputs)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

# FCN1 模型
class FCN1(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_units=16):
        super(FCN1, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(num_inputs, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, num_outputs)
        )

    def forward(self, x):
        return self.net(x)

# FCN2 模型
class FCN2(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_units1=32, hidden_units2=16):
        super(FCN2, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(num_inputs, hidden_units1),
            nn.ReLU(),
            nn.Linear(hidden_units1, hidden_units2),
            nn.ReLU(),
            nn.Linear(hidden_units2, num_outputs)
        )

    def forward(self, x):
        return self.net(x)

# 定义损失函数
loss_func = nn.MSELoss()

# 超参数范围
learning_rates = [0.01, 0.005, 0.001]
batch_sizes = [128, 64, 32]

# K折交叉验证
k = 3
kf = KFold(n_splits=k, shuffle=True, random_state=42)


In [3]:
# 超参数范围
learning_rates = [0.01, 0.005, 0.001]
batch_sizes = [128, 64, 32]
num_epochs_list = [50, 100]
hidden_units_list = [16, 32]

def create_time_series_data(data, time_step):
    num_samples = data.shape[0]
    num_features = data.shape[1]
    time_series_data = []

    for i in range(num_samples - time_step + 1):
        time_series_data.append(data[i:i + time_step].clone().detach().numpy())

    return torch.tensor(np.array(time_series_data))

def train_and_evaluate_model(model_class, num_inputs, num_outputs, train_input_tensor, train_label_tensor, time_step=24):
    results = []
    for lr in learning_rates:
        for batch_size in batch_sizes:
            for num_epochs in num_epochs_list:
                for hidden_units in hidden_units_list:
                    fold_results = []
                    for fold, (train_index, val_index) in enumerate(kf.split(train_input_tensor)):
                        # 创建模型实例
                        if model_class in [CNN, CNNLSTM]:
                            model = model_class(num_inputs, num_outputs, hidden_units, time_step)
                        else:
                            model = model_class(num_inputs, num_outputs, hidden_units)

                        optimizer = optim.Adam(model.parameters(), lr=lr)

                        # 分割数据
                        train_input_fold = train_input_tensor[train_index]
                        train_label_fold = train_label_tensor[train_index]
                        val_input_fold = train_input_tensor[val_index]
                        val_label_fold = train_label_tensor[val_index]

                        # 如果模型是CNN或LSTM，转换数据为时间序列格式
                        if issubclass(model_class, (CNN, LSTM, CNNLSTM)):
                            train_input_fold = create_time_series_data(train_input_fold, time_step)
                            train_label_fold = train_label_fold[time_step - 1:]  # 对齐标签
                            val_input_fold = create_time_series_data(val_input_fold, time_step)
                            val_label_fold = val_label_fold[time_step - 1:]  # 对齐标签

                        # 创建数据加载器
                        train_loader = torch.utils.data.DataLoader(
                            dataset=torch.utils.data.TensorDataset(train_input_fold.clone().detach(), train_label_fold.clone().detach()),
                            batch_size=batch_size,
                            shuffle=True
                        )
                        val_loader = torch.utils.data.DataLoader(
                            dataset=torch.utils.data.TensorDataset(val_input_fold.clone().detach(), val_label_fold.clone().detach()),
                            batch_size=batch_size,
                            shuffle=False
                        )

                        # 训练模型
                        start_time = time.time()
                        for epoch in range(num_epochs):
                            model.train()
                            for step, (x, y) in enumerate(train_loader):
                                output = model(x)
                                loss = loss_func(output, y)
                                optimizer.zero_grad()
                                loss.backward()
                                optimizer.step()

                        # 验证模型
                        model.eval()
                        val_loss = 0
                        with torch.no_grad():
                            for x, y in val_loader:
                                output = model(x)
                                loss = loss_func(output, y)
                                val_loss += loss.item()
                        val_loss /= len(val_loader)
                        fold_results.append(val_loss)
                        end_time = time.time()
                        print(f"Fold {fold + 1}, LR: {lr}, Batch Size: {batch_size}, Epochs: {num_epochs}, Hidden Units: {hidden_units}, Validation Loss: {val_loss:.4f}, Time: {end_time - start_time:.2f} seconds")

                    # 计算当前超参数组合在所有折上的平均损失
                    avg_val_loss = np.mean(fold_results)
                    results.append({
                        'learning_rate': lr,
                        'batch_size': batch_size,
                        'num_epochs': num_epochs,
                        'hidden_units': hidden_units,
                        'avg_val_loss': avg_val_loss
                    })

    # 转换结果为DataFrame以便分析
    results_df = pd.DataFrame(results)

    # 找到最佳配置
    best_config = results_df.loc[results_df['avg_val_loss'].idxmin()]

    print(f"\nBest Configuration for {model_class.__name__}: {best_config}")
    return results_df
      
# 评估每个模型
cnn_results = train_and_evaluate_model(CNN, num_inputs=10, num_outputs=1, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor)
lstm_results = train_and_evaluate_model(LSTM, num_inputs=10, num_outputs=1, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor)
cnn_lstm_results = train_and_evaluate_model(CNNLSTM, num_inputs=10, num_outputs=1, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor)
transformer_results = train_and_evaluate_model(Transformer, num_inputs=10, num_outputs=1, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor)
fcn1_results = train_and_evaluate_model(FCN1, num_inputs=10, num_outputs=1, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor)
fcn2_results = train_and_evaluate_model(FCN2, num_inputs=10, num_outputs=1, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor)

Fold 1, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 16, Validation Loss: 0.5897, Time: 1.23 seconds
Fold 2, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 16, Validation Loss: 0.9388, Time: 6.42 seconds
Fold 3, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 16, Validation Loss: 0.6818, Time: 7.44 seconds
Fold 1, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 32, Validation Loss: 0.5116, Time: 6.77 seconds
Fold 2, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 32, Validation Loss: 0.8190, Time: 5.79 seconds
Fold 3, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 32, Validation Loss: 0.5931, Time: 6.20 seconds
Fold 1, LR: 0.01, Batch Size: 128, Epochs: 100, Hidden Units: 16, Validation Loss: 0.6236, Time: 15.26 seconds
Fold 2, LR: 0.01, Batch Size: 128, Epochs: 100, Hidden Units: 16, Validation Loss: 0.8376, Time: 14.31 seconds
Fold 3, LR: 0.01, Batch Size: 128, Epochs: 100, Hidden Units: 16, Validation Loss: 0.7775, Time: 15.02 seconds
Fold 1, LR: 0

C:\Users\23124862r\AppData\Local\anaconda3\envs\kan\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Fold 1, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 16, Validation Loss: 0.1796, Time: 25.21 seconds
Fold 2, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 16, Validation Loss: 0.2163, Time: 26.78 seconds
Fold 3, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 16, Validation Loss: 0.2202, Time: 22.27 seconds
Fold 1, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 32, Validation Loss: 0.3579, Time: 23.25 seconds
Fold 2, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 32, Validation Loss: 0.3534, Time: 26.25 seconds
Fold 3, LR: 0.01, Batch Size: 128, Epochs: 50, Hidden Units: 32, Validation Loss: 0.7669, Time: 30.31 seconds
Fold 1, LR: 0.01, Batch Size: 128, Epochs: 100, Hidden Units: 16, Validation Loss: 0.1372, Time: 43.83 seconds
Fold 2, LR: 0.01, Batch Size: 128, Epochs: 100, Hidden Units: 16, Validation Loss: 0.1096, Time: 44.13 seconds
Fold 3, LR: 0.01, Batch Size: 128, Epochs: 100, Hidden Units: 16, Validation Loss: 0.1110, Time: 53.10 seconds
Fold 1,

In [4]:
results_df = pd.DataFrame(cnn_results)
results_df.to_csv('./parameters_configuration/cnn_BaseConfig-SPRING.csv') 
results_df = pd.DataFrame(lstm_results)
results_df.to_csv('./parameters_configuration/lstm_BaseConfig-SPRING.csv') 
results_df = pd.DataFrame(cnn_lstm_results)
results_df.to_csv('./parameters_configuration/cnn_lstm_BaseConfig-SPRING.csv') 
results_df = pd.DataFrame(transformer_results)
results_df.to_csv('./parameters_configuration/transformer_BaseConfig-SPRING.csv') 
results_df = pd.DataFrame(fcn1_results)
results_df.to_csv('./parameters_configuration/fcn1_BaseConfig-SPRING.csv') 
results_df = pd.DataFrame(fcn2_results)
results_df.to_csv('./parameters_configuration/fcn2_BaseConfig-SPRING.csv') 

In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

results_df_cnn = pd.read_csv('./parameters_configuration/cnn_BaseConfig-SPRING.csv',index_col=0)
results_df_lstm = pd.read_csv('./parameters_configuration/lstm_BaseConfig-SPRING.csv',index_col=0)
results_df_cnn_lstm = pd.read_csv('./parameters_configuration/cnn_lstm_BaseConfig-SPRING.csv',index_col=0)
results_df_transformer = pd.read_csv('./parameters_configuration/transformer_BaseConfig-SPRING.csv',index_col=0)
results_df_fcn1 = pd.read_csv('./parameters_configuration/fcn1_BaseConfig-SPRING.csv',index_col=0)
results_df_fcn2 = pd.read_csv('./parameters_configuration/fcn2_BaseConfig-SPRING.csv',index_col=0)
def train_and_test_model(model_class, learning_rate, batch_size, num_epochs, hidden_units, train_input_tensor, train_label_tensor, test_input_tensor, test_label_tensor, scaler_output, num_inputs=10, num_outputs=1, time_step=24):
    results = []

    # 如果模型是CNN或LSTM，转换数据为时间序列格式
    if issubclass(model_class, (CNN, LSTM, CNNLSTM)):
        train_input_tensor = create_time_series_data(train_input_tensor, time_step)
        train_label_tensor = train_label_tensor[time_step - 1:]  # 对齐标签
        test_input_tensor = create_time_series_data(test_input_tensor, time_step)
        test_label_tensor = test_label_tensor[time_step - 1:]  # 对齐标签

    # 创建数据加载器
    train_loader = torch.utils.data.DataLoader(
        dataset=torch.utils.data.TensorDataset(train_input_tensor.clone().detach(), train_label_tensor.clone().detach()),
        batch_size=int(batch_size),
        shuffle=True
    )

    # 初始化模型、损失函数和优化器
    model = model_class(num_inputs, num_outputs, hidden_units)
    loss_func = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 训练模型
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        for step, (x, y) in enumerate(train_loader):
            output = model(x)
            loss = loss_func(output, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # 验证模型
    model.eval()
    with torch.no_grad():
        output = model(test_input_tensor.clone().detach())
        output_inverse = scaler_output.inverse_transform(output.numpy())
        test_label_inverse = scaler_output.inverse_transform(test_label_tensor.numpy())

        # 计算评估指标
        rmse = np.sqrt(mean_squared_error(test_label_inverse, output_inverse))
        mae = mean_absolute_error(test_label_inverse, output_inverse)
        mape = np.mean(np.abs((test_label_inverse - output_inverse) / test_label_inverse)) * 100

        test_loss = loss_func(output, test_label_tensor.clone().detach()).item()

    # print(f"Test Loss: {test_loss:.4f}")

    results.append({
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape
    })

    return results

In [7]:
# 找到每个模型的最佳配置
def get_best_config(results_df):
    best_config = results_df.loc[results_df['avg_val_loss'].idxmin(), ['learning_rate', 'batch_size', 'num_epochs', 'hidden_units']]
    return {
        'learning_rate': best_config['learning_rate'],
        'batch_size': int(best_config['batch_size']),
        'num_epochs': int(best_config['num_epochs']),
        'hidden_units': int(best_config['hidden_units'])
    }

# 获取每个模型的最佳配置
best_config_cnn = get_best_config(results_df_cnn)
best_config_lstm = get_best_config(results_df_lstm)
best_config_cnn_lstm = get_best_config(results_df_cnn_lstm)
best_config_transformer = get_best_config(results_df_transformer)
best_config_fcn1 = get_best_config(results_df_fcn1)
best_config_fcn2 = get_best_config(results_df_fcn2)

# 假设 train_input_tensor, train_label_tensor, test_input_tensor, test_label_tensor 已经定义

# 训练和测试每个模型，并打印结果
cnn_test_results = train_and_test_model(CNN, **best_config_cnn, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor, test_input_tensor=test_input_tensor, test_label_tensor=test_label_tensor, scaler_output=scaler_output)
print("CNN Test Results:", cnn_test_results)

lstm_test_results = train_and_test_model(LSTM, **best_config_lstm, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor, test_input_tensor=test_input_tensor, test_label_tensor=test_label_tensor, scaler_output=scaler_output)
print("LSTM Test Results:", lstm_test_results)

cnn_lstm_test_results = train_and_test_model(CNNLSTM, **best_config_cnn_lstm, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor, test_input_tensor=test_input_tensor, test_label_tensor=test_label_tensor, scaler_output=scaler_output)
print("CNN-LSTM Test Results:", cnn_lstm_test_results)

transformer_test_results = train_and_test_model(Transformer, **best_config_transformer, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor, test_input_tensor=test_input_tensor, test_label_tensor=test_label_tensor, scaler_output=scaler_output)
print("Transformer Test Results:", transformer_test_results)

fcn1_test_results = train_and_test_model(FCN1, **best_config_fcn1, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor, test_input_tensor=test_input_tensor, test_label_tensor=test_label_tensor, scaler_output=scaler_output)
print("FCN1 Test Results:", fcn1_test_results)

fcn2_test_results = train_and_test_model(FCN2, **best_config_fcn2, train_input_tensor=train_input_tensor, train_label_tensor=train_label_tensor, test_input_tensor=test_input_tensor, test_label_tensor=test_label_tensor, scaler_output=scaler_output)
print("FCN2 Test Results:", fcn2_test_results)

CNN Test Results: [{'RMSE': 363.12497, 'MAE': 284.53516, 'MAPE': 3.8278382271528244}]
LSTM Test Results: [{'RMSE': 374.975, 'MAE': 295.3712, 'MAPE': 3.956039622426033}]
CNN-LSTM Test Results: [{'RMSE': 382.98364, 'MAE': 304.5893, 'MAPE': 4.069294407963753}]


C:\Users\23124862r\AppData\Local\anaconda3\envs\kan\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Transformer Test Results: [{'RMSE': 395.73175, 'MAE': 282.6444, 'MAPE': 3.799699991941452}]
FCN1 Test Results: [{'RMSE': 379.38968, 'MAE': 307.85696, 'MAPE': 4.097015410661697}]
FCN2 Test Results: [{'RMSE': 367.3222, 'MAE': 288.25598, 'MAPE': 3.80210317671299}]


In [11]:
result = dict()
result["cnn"] = cnn_test_results
result["lstm"] = lstm_test_results
result["cnn_lstm"] = cnn_lstm_test_results
result["transformer"] = transformer_test_results
result["fcn1"] = fcn1_test_results
result["fcn2"] = fcn2_test_results

In [12]:
result

{'cnn': [{'RMSE': 363.12497, 'MAE': 284.53516, 'MAPE': 3.8278382271528244}],
 'lstm': [{'RMSE': 374.975, 'MAE': 295.3712, 'MAPE': 3.956039622426033}],
 'cnn_lstm': [{'RMSE': 382.98364, 'MAE': 304.5893, 'MAPE': 4.069294407963753}],
 'transformer': [{'RMSE': 395.73175,
   'MAE': 282.6444,
   'MAPE': 3.799699991941452}],
 'fcn1': [{'RMSE': 379.38968, 'MAE': 307.85696, 'MAPE': 4.097015410661697}],
 'fcn2': [{'RMSE': 367.3222, 'MAE': 288.25598, 'MAPE': 3.80210317671299}]}